In [1]:
%load_ext autoreload
%autoreload 2

# deepOF model evaluation

Given a dataset and a trained model, this notebook allows the user to 

* Load and inspect the different models (encoder, decoder, grouper, gmvaep)
* Visualize reconstruction quality for a given model
* Visualize a static latent space
* Visualize trajectories on the latent space for a given video
* sample from the latent space distributions and generate video clips showcasing generated data

In [2]:
import os
os.chdir(os.path.dirname("../"))

In [15]:
import deepof.data
import deepof.utils
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

from ipywidgets import interact
from IPython import display
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import seaborn as sns

from ipywidgets import interact

### 1. Define and run project

In [4]:
path = os.path.join("..", "..", "Desktop", "deepoftesttemp")
trained_network = os.path.join("..", "..", "Desktop")

In [5]:
%%time
proj = deepof.data.project(
    path=path,
    smooth_alpha=0.99,
    exclude_bodyparts=["Tail_1", "Tail_2", "Tail_tip", "Tail_base"],
    arena_dims=[380],
)

CPU times: user 297 ms, sys: 23.9 ms, total: 321 ms
Wall time: 270 ms


In [6]:
%%time
proj = proj.run(verbose=True)
print(proj)

Loading trajectories...
Smoothing trajectories...
Interpolating outliers...
Iterative imputation of ocluded bodyparts...
Computing distances...
Computing angles...
Done!
deepof analysis of 2 videos
CPU times: user 2.63 s, sys: 86.3 ms, total: 2.72 s
Wall time: 683 ms


### 2. Load pretrained deepof model

In [7]:
coords = proj.get_coords(center="Center", align="Spine_1", align_inplace=True)
preprocessed_data,_,_,_ = coords.preprocess(test_videos=0, window_step=5, window_size=24)

In [8]:
# Set model parameters
encoding=6
loss="ELBO"
k=25
pheno=0
predictor=0

In [10]:
encoder, decoder, grouper, gmvaep = deepof.models.SEQ_2_SEQ_GMVAE(
    loss=loss,
    number_of_components=k,
    compile_model=True,
    encoding=encoding,
    predictor=predictor,
    phenotype_prediction=pheno,
).build(preprocessed_data.shape)[:4]

gmvaep.load_weights(
    os.path.join(
        trained_network, [i for i in os.listdir(trained_network) if i.endswith("h5")][0]
    )
)

In [11]:
# Uncomment to see model summaries
# encoder.summary()
# decoder.summary()
# grouper.summary()
# gmvaep.summary()

In [12]:
# Uncomment to plot model structure
def plot_model(model, name):
    tf.keras.utils.plot_model(
        model,
        to_file=os.path.join(
            path,
            "deepof_{}_{}.png".format(name, datetime.now().strftime("%Y%m%d-%H%M%S")),
        ),
        show_shapes=True,
        show_dtype=False,
        show_layer_names=True,
        rankdir="TB",
        expand_nested=True,
        dpi=200,
    )


# plot_model(encoder, "encoder")
# plot_model(decoder, "decoder")
# plot_model(grouper, "grouper")
# plot_model(gmvaep, "gmvaep")

### 4. Pass data through all models

In [13]:
encodings = encoder.predict(preprocessed_data)
groupings = grouper.predict(preprocessed_data)
reconstrs = gmvaep.predict(preprocessed_data)

### 5. Evaluate reconstruction (to be incorporated into deepof.evaluate)

In [16]:
# Fit a scaler to the data, to back-transform reconstructions later
scaler = StandardScaler().fit(preprocessed_data[:, 0, :])

In [17]:
# Rescale reconstructions
rescaled_reconstructions = scaler.transform(
    reconstrs.reshape(reconstrs.shape[0] * reconstrs.shape[1], reconstrs.shape[2])
)
rescaled_reconstructions = rescaled_reconstructions.reshape(reconstrs.shape)

In [ ]:
# Display a video with the original data superimposed with the reconstructions


### 6. Evaluate latent space (to be incorporated into deepof.evaluate)